In [18]:
import pandas as pd
import numpy as np
import datetime as dt
from collections import defaultdict 
from copy import deepcopy

In [14]:
rawdata = pd.read_csv("order_brush_order.csv")
print(rawdata.describe())
rawdata.head(10)

            orderid        shopid        userid
count  2.227500e+05  2.227500e+05  2.227500e+05
mean   3.130027e+13  9.433117e+07  9.802880e+07
std    1.222774e+11  5.695790e+07  6.839048e+07
min    3.107520e+13  1.000900e+04  1.000700e+04
25%    3.120360e+13  4.980267e+07  3.508127e+07
50%    3.130561e+13  9.033636e+07  9.309625e+07
75%    3.140604e+13  1.475053e+08  1.590612e+08
max    3.150720e+13  2.154352e+08  2.155262e+08


,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
5,31119725718155,173811070,116451780,2019-12-27 12:22:05
6,31151322178251,107921853,166741763,2019-12-27 21:08:43
7,31079856153738,178400128,61272835,2019-12-27 01:17:37
8,31118426867571,147941492,10986763,2019-12-27 12:00:27
9,31133458226149,164933170,135957741,2019-12-27 16:10:59


In [15]:
data_temp = deepcopy(rawdata)
data_temp['min_time'] = data_temp['event_time'].apply(lambda x: dt.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
data_temp['max_time'] = data_temp['min_time'].apply(lambda x: x + dt.timedelta(hours =1))
data_temp['time_range'] = data_temp['min_time'].apply(lambda x: (x,x + dt.timedelta(hours =1)))
rawdata['event_time'] = pd.to_datetime(rawdata['event_time'])
data_temp['shopid'] =data_temp['orderid'].astype(str)
data_temp['shopid'] =data_temp['shopid'].astype(str)
data_temp['userid'] =data_temp['userid'].astype(str)
data_temp.head()

,orderid,shopid,userid,event_time,min_time,max_time,time_range
0,31076582227611,31076582227611,30530270,2019-12-27 00:23:03,2019-12-27 00:23:03,2019-12-27 01:23:03,"(2019-12-27 00:23:03, 2019-12-27 01:23:03)"
1,31118059853484,31118059853484,46057927,2019-12-27 11:54:20,2019-12-27 11:54:20,2019-12-27 12:54:20,"(2019-12-27 11:54:20, 2019-12-27 12:54:20)"
2,31123355095755,31123355095755,67341739,2019-12-27 13:22:35,2019-12-27 13:22:35,2019-12-27 14:22:35,"(2019-12-27 13:22:35, 2019-12-27 14:22:35)"
3,31122059872723,31122059872723,149380322,2019-12-27 13:01:00,2019-12-27 13:01:00,2019-12-27 14:01:00,"(2019-12-27 13:01:00, 2019-12-27 14:01:00)"
4,31117075665123,31117075665123,149493217,2019-12-27 11:37:55,2019-12-27 11:37:55,2019-12-27 12:37:55,"(2019-12-27 11:37:55, 2019-12-27 12:37:55)"


In [22]:
import csv
import bisect
import operator
from datetime import datetime


def writeOutput():
    global key, value
    with open('output.csv', mode='w') as output_file:
        writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['ShopId', 'UserId'])
        for key, value in result.items():
            writer.writerow([key, value])


def buildOutput():
    global sorted_dict, i
    if len(outputUsers) == 0:
        result[shopId] = 0
    else:
        sorted_dict = sorted(outputUsers.items(), key=operator.itemgetter(1))
        sorted_dict.reverse()
        s = ""
        i = -1
        for u in sorted_dict:
            i = i + 1
            if i > 0:
                s = s + "&"
            s = s + u[0]
        result[shopId] = s


def storeSuspiciousUsers():
    global sorted_dict
    sorted_dict = sorted(dict.items(), key=operator.itemgetter(1))
    sorted_dict.reverse()
    max = sorted_dict[0][1]
    for user in sorted_dict:
        if user[1] == max:
            if not user[0] in outputUsers or outputUsers[user[0]] < user[1]:
                outputUsers[user[0]] = user[1]
        else:
            break


def readInput():
    global row, orderId, shopId, userId, eventTime
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            orderId = row[0]
            shopId = row[1]
            userId = row[2]
            eventTime = row[3]
            if not shopId in dict:
                dict[shopId] = []
            bisect.insort(dict[shopId], (datetime.strptime(eventTime, FORMAT), userId, orderId))
            line_count += 1


def scanTransactions():
    global key, value, startTime
    numberOfUser = len(dict)
    numberOfOrder = 0
    for key, value in dict.items():
        numberOfOrder += value
    if numberOfOrder / numberOfUser >= 3:
        storeSuspiciousUsers()
    firstTuple = queue.pop(0)
    if firstTuple[1] in dict:
        dict[firstTuple[1]] = dict[firstTuple[1]] - 1
    if len(queue) > 0:
        startTime = queue[0][0]


with open("order_brush_order.csv") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    dict = {}
    FORMAT = "%Y-%m-%d %H:%M:%S"
    LIMIT_DURATION = 3600
    result = {}
    readInput()
    for shopId, values in dict.items():
        i = -1
        j = 0
        startTime = 0
        queue = []
        dict = {}
        outputUsers = {}
        for v in values:
            i = i + 1
            eventTime = v[0]
            userId = v[1]
            orderId = v[2]
            if i == 0:
                startTime = eventTime
            duration = eventTime - startTime
            duration = duration.total_seconds()
            queue.append(v)
            if duration <= LIMIT_DURATION:
                if not userId in dict:
                    dict[userId] = 0
                dict[userId] = dict[userId] + 1
            if duration > LIMIT_DURATION or i == len(values) - 1:
                scanTransactions()
        buildOutput()
    writeOutput()
    print(f'Processed {line_count} lines.')



Column names are orderid, shopid, userid, event_time
Processed 0 lines.


In [23]:
import csv
import bisect
import operator
from datetime import datetime

with open("order_brush_order.csv") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    dict = {}
    FORMAT = "%Y-%m-%d %H:%M:%S"
    LIMIT_DURATION = 3600
    result = {}

    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        elif len(row) > 0:
            orderId = row[0]
            shopId = row[1]
            userId = row[2]
            eventTime = row[3]
            if not shopId in dict:
                dict[shopId] = []
            bisect.insort(dict[shopId], (datetime.strptime(eventTime, FORMAT), userId, orderId))
            line_count += 1
    for shopId, values in dict.items():
        outputUsers = {}
        for i in range(0, len(values)):
            duration = 0
            dict1 = {}
            for j in range(i, len(values)):
                duration = duration + (values[j][0] - values[i][0]).total_seconds()
                userId = values[j][1]
                if not userId in dict1:
                    dict1[userId] = 0
                dict1[userId] = dict1[userId] + 1
                if duration <= LIMIT_DURATION:
                    numberOfUser = len(dict1)
                    numberOfOrder = 0
                    for key, value in dict1.items():
                        numberOfOrder += value
                    if numberOfUser == 0:
                        isAppend = True
                        continue
                    if numberOfOrder / numberOfUser >= 3:
                        sorted_dict = sorted(dict1.items(), key=operator.itemgetter(1))
                        sorted_dict.reverse()
                        max = sorted_dict[0][1]
                        for user in sorted_dict:
                            if user[1] == max:
                                if not user[0] in outputUsers or outputUsers[user[0]] < user[1]:
                                    outputUsers[user[0]] = user[1]
                            else:
                                break
        if len(outputUsers) == 0:
            result[shopId] = 0
        else:
            sorted_dict = sorted(outputUsers.items(), key=operator.itemgetter(1))
            sorted_dict.reverse()
            s = ""
            i = -1
            for u in sorted_dict:
                i = i + 1
                if i > 0:
                    s = s + "&"
                s = s + u[0]
            result[shopId] = s
    with open('output1_Hoa.csv', mode='w') as output_file:
        writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['ShopId', 'UserId'])
        for key, value in result.items():
            writer.writerow([key, value])
    print(f'Processed {line_count} lines.')


Column names are orderid, shopid, userid, event_time
Processed 222751 lines.


In [24]:
df = pd.read_csv('output1_Hoa.csv')
len(df)

18770